In [3]:
from Process import PlateReader
import pandas as pd

### Step 1: Load Sample Information
* Ideally will be in the form of a dataframe. This dataframe should be standardized in the original documentation where we originally made the sample. 
* Either load locally or pull from the google team drive (still working on fixing pulling from Drive)

In [4]:
sample_info = pd.read_csv(r"04_16_21_Yt")

FileNotFoundError: [Errno 2] No such file or directory: '04_16_21_Yt'

### Step 2: Load UV-Vis Data
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
         
* Sometimes when using the plate reader the scanning protocol scans the whole plate versus only the wells of interest. This can be changed, however it is common to forget. If you do forget to do this, make sure to trim the dataframe to match the same length/order of wells as the sample info loaded in Step 1.

* ***If loading from single cuvette reader***: 
    * Still documenting.

***Step 2a:*** Provide the path of the excel sheet for the plate reader. With this you will also need to provide the **Sheet Name** of for correct excel sheet to be extracted. Since multiple plates are expected to be used at once, provide the Sheet names as a ***list of string names***. This will return a list of dataframes of each plate in the order the sheet names provided.

In [5]:
path = r"04_16_21_Yt.xlsx"
plate_names = ['Sheet1']
plate_dfs = PlateReader.extract_plates(path, plate_names)

***Step 2b:*** With the dataframes from ***Step2a***, given that all the plates read originally were done using the same protcol, then we can merge these dataframes into one. The only thing we need to edit is when originally formatting the excel sheets, since we copied the wavelength row in each sheet it only needs to be present once. A way to avoid to this is to copy it only in the first plate sheet you are using. 

Should be length of sample_info + 1 (will remove the wavelength column shortly).

In [8]:
merged_df = PlateReader.merge_wavelength_dfs(plate_dfs)

***Step 2c:*** Sometimes when samples are extremely turbid or have some sort of material like foam or cream the plate reader absorbance is overflowed and returns as 'OVRFLW'. This will prevent you from handling the entire set of data, so what can be done is replace the 'OVRFLW' values with a holder number. It is reccomended you select a very high holder number so you can track and isolate it easier if needed. 

In [9]:
merged_df = PlateReader.detect_ovflw(merged_df, holder = 30)
merged_df

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
Wavelength,300.000,305.000,310.000,315.000,320.000,325.000,330.000,335.000,340.000,345.000,...,655.000,660.000,665.000,670.000,675.000,680.000,685.000,690.000,695.000,700.000
A1,0.857,0.700,0.636,0.589,0.537,0.495,0.459,0.423,0.393,0.373,...,0.058,0.058,0.058,0.058,0.058,0.057,0.058,0.058,0.058,0.058
A2,0.827,0.675,0.608,0.559,0.507,0.465,0.429,0.392,0.363,0.343,...,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037
A3,0.890,0.738,0.671,0.623,0.571,0.529,0.493,0.458,0.428,0.409,...,0.095,0.095,0.094,0.094,0.093,0.093,0.092,0.092,0.092,0.092
A4,0.885,0.735,0.669,0.621,0.571,0.528,0.493,0.457,0.427,0.408,...,0.095,0.094,0.094,0.093,0.093,0.092,0.092,0.092,0.091,0.091
A5,1.072,0.933,0.866,0.815,0.763,0.719,0.681,0.643,0.612,0.587,...,0.134,0.133,0.131,0.130,0.128,0.127,0.126,0.124,0.123,0.122
A6,1.109,0.952,0.881,0.829,0.774,0.727,0.687,0.647,0.613,0.589,...,0.126,0.124,0.123,0.122,0.120,0.119,0.118,0.117,0.116,0.115
A7,1.211,1.043,0.964,0.905,0.845,0.793,0.749,0.705,0.668,0.640,...,0.136,0.135,0.133,0.132,0.130,0.129,0.128,0.127,0.125,0.124
A8,2.172,1.956,1.822,1.700,1.574,1.460,1.358,1.262,1.178,1.108,...,0.215,0.212,0.210,0.207,0.204,0.202,0.200,0.197,0.195,0.193


# Step 3: Merge with Sample Info Dataframe

The first step to merge to sample info dataframe is to format the plate reader dataframe to have the wavelengths as the column names versus as an another row. We can do this by isolating the Wavelength row, adding the appropiate unit the string of each element of the wavlength row and reassigning this new list as the column names. We finalize this by dropping the wavlength row.

In [36]:
platereader_df = PlateReader.rehead_wavelengths(merged_df)
platereader_df = platereader_df-platereader_df.iloc[0]

In [37]:
platereader_df.insert(0, 'Sonication Time (Min)', [0,0,10,10,20,20,30,30])
platereader_df

,Sonication Time (Min),300.0nm,305.0nm,310.0nm,315.0nm,320.0nm,325.0nm,330.0nm,335.0nm,340.0nm,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
A1,0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
A2,0,-0.030,-0.025,-0.028,-0.030,-0.030,-0.030,-0.030,-0.031,-0.030,...,-0.021,-0.021,-0.021,-0.021,-0.021,-0.020,-0.021,-0.021,-0.021,-0.021
A3,10,0.033,0.038,0.035,0.034,0.034,0.034,0.034,0.035,0.035,...,0.037,0.037,0.036,0.036,0.035,0.036,0.034,0.034,0.034,0.034
A4,10,0.028,0.035,0.033,0.032,0.034,0.033,0.034,0.034,0.034,...,0.037,0.036,0.036,0.035,0.035,0.035,0.034,0.034,0.033,0.033
A5,20,0.215,0.233,0.230,0.226,0.226,0.224,0.222,0.220,0.219,...,0.076,0.075,0.073,0.072,0.070,0.070,0.068,0.066,0.065,0.064
A6,20,0.252,0.252,0.245,0.240,0.237,0.232,0.228,0.224,0.220,...,0.068,0.066,0.065,0.064,0.062,0.062,0.060,0.059,0.058,0.057
A7,30,0.354,0.343,0.328,0.316,0.308,0.298,0.290,0.282,0.275,...,0.078,0.077,0.075,0.074,0.072,0.072,0.070,0.069,0.067,0.066
A8,30,1.315,1.256,1.186,1.111,1.037,0.965,0.899,0.839,0.785,...,0.157,0.154,0.152,0.149,0.146,0.145,0.142,0.139,0.137,0.135


In [20]:
spectras = platereader_df[[col for col in platereader_df.columns if 'nm' in col]]

NameError: name 'platereader_df_spectras' is not defined

In [38]:
platereader_df.to_csv('04_16_21_Yt_Dataframe_Blanked')

In [32]:
platereader_df

,Sonication Time (Min),300.0nm,305.0nm,310.0nm,315.0nm,320.0nm,325.0nm,330.0nm,335.0nm,340.0nm,...,655.0nm,660.0nm,665.0nm,670.0nm,675.0nm,680.0nm,685.0nm,690.0nm,695.0nm,700.0nm
A1,0,0.857,0.700,0.636,0.589,0.537,0.495,0.459,0.423,0.393,...,0.058,0.058,0.058,0.058,0.058,0.057,0.058,0.058,0.058,0.058
A2,0,0.827,0.675,0.608,0.559,0.507,0.465,0.429,0.392,0.363,...,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037,0.037
A3,10,0.890,0.738,0.671,0.623,0.571,0.529,0.493,0.458,0.428,...,0.095,0.095,0.094,0.094,0.093,0.093,0.092,0.092,0.092,0.092
A4,10,0.885,0.735,0.669,0.621,0.571,0.528,0.493,0.457,0.427,...,0.095,0.094,0.094,0.093,0.093,0.092,0.092,0.092,0.091,0.091
A5,20,1.072,0.933,0.866,0.815,0.763,0.719,0.681,0.643,0.612,...,0.134,0.133,0.131,0.130,0.128,0.127,0.126,0.124,0.123,0.122
A6,20,1.109,0.952,0.881,0.829,0.774,0.727,0.687,0.647,0.613,...,0.126,0.124,0.123,0.122,0.120,0.119,0.118,0.117,0.116,0.115
A7,30,1.211,1.043,0.964,0.905,0.845,0.793,0.749,0.705,0.668,...,0.136,0.135,0.133,0.132,0.130,0.129,0.128,0.127,0.125,0.124
A8,30,2.172,1.956,1.822,1.700,1.574,1.460,1.358,1.262,1.178,...,0.215,0.212,0.210,0.207,0.204,0.202,0.200,0.197,0.195,0.193


Next we reset both indexes of the two dataframes we are to merge and concatnate them together. We drop and reset indexes to ensure any past dependency is forgotten, so you must make sure your sample info dataframe and plate reader dataframe are ordered correctly where row 1...n of one dataframe referes to row 1...n of the other. 

In [19]:
# sample_info_df.reset_index(drop=True, inplace=True)
# platereader_df.reset_index(drop=True, inplace=True)
# combined_df = pd.concat([sample_info_df, platereader_df], axis = 1) # this is called combine hotdog. 
# combined_df

,Total Volume,CdOl 84mM volpct,TOP 1mM volpct,ZnOl 84mM volpct,Time Sonicated,200nm,205nm,210nm,215nm,220nm,...,855nm,860nm,865nm,870nm,875nm,880nm,885nm,890nm,895nm,900nm
0,5.0,75.0,25.0,0.0,0.0,3.497,3.566,3.545,3.519,3.57,...,0.048,0.048,0.051,0.057,0.062,0.061,0.058,0.059,0.063,0.065
1,5.0,75.0,25.0,0.0,0.0,3.49,3.551,3.538,3.512,3.564,...,0.042,0.042,0.045,0.051,0.057,0.055,0.053,0.054,0.058,0.06
2,5.0,75.0,24.5,0.5,0.0,3.544,3.608,3.597,3.57,3.615,...,0.041,0.041,0.044,0.05,0.056,0.055,0.052,0.054,0.058,0.06
3,5.0,75.0,24.5,0.5,0.0,3.533,3.602,3.583,3.561,3.61,...,0.042,0.042,0.045,0.051,0.058,0.057,0.054,0.056,0.06,0.062
4,5.0,75.0,24.0,1.0,0.0,3.524,3.594,3.577,3.557,3.601,...,0.044,0.044,0.047,0.053,0.06,0.058,0.056,0.057,0.061,0.064
5,5.0,75.0,24.0,1.0,0.0,3.535,3.61,3.591,3.568,3.618,...,0.043,0.043,0.046,0.052,0.059,0.057,0.055,0.056,0.06,0.063
6,5.0,75.0,20.0,5.0,0.0,3.539,3.61,3.59,3.569,3.618,...,0.04,0.04,0.043,0.049,0.056,0.054,0.052,0.053,0.057,0.059
7,5.0,75.0,20.0,5.0,0.0,3.543,3.617,3.6,3.574,3.622,...,0.041,0.041,0.044,0.05,0.057,0.055,0.053,0.054,0.058,0.061
8,5.0,75.0,25.0,0.0,10.0,3.641,3.7,3.693,3.678,3.722,...,0.043,0.043,0.046,0.052,0.058,0.057,0.055,0.056,0.06,0.062
9,5.0,75.0,25.0,0.0,10.0,3.608,3.667,3.661,3.64,3.686,...,0.039,0.04,0.042,0.048,0.054,0.052,0.051,0.052,0.056,0.058


# Step 4: Format Dataframe as csv and Upload to Drive.
We simply call a method of the dataframe while ensuring to drop the index (again to remove and previous dependency). 

***If uploading from desktop***: Just grab and upload like any other file.

***If uploading from jupyter notebook***: Save the csv anywhere with a name that is not relevant, reccomended is something like "Temp File" as you will be uploading it to google drive under a different name. Then use the Gdrive module of functions from the Prepare package. Read the README in this package to understand how to use it, it will require a certain setup of credentials. 


In [32]:
temp_hold_path = r"04_09_21 Cadmium Oleate HT Blanked"
combined_df_csv = combined_df.to_csv(temp_hold_path, index=False)

,Total Volume,CdOl 84mM volpct,TOP 1mM volpct,ZnOl 84mM volpct,Time Sonicated,200nm,205nm,210nm,215nm,220nm,...,855nm,860nm,865nm,870nm,875nm,880nm,885nm,890nm,895nm,900nm
0,5.0,75.0,25.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,5.0,75.0,25.0,0.0,0.0,-0.007,-0.015,-0.007,-0.007,-0.006,...,-0.006,-0.006,-0.006,-0.006,-0.005,-0.006,-0.005,-0.005,-0.005,-0.005
2,5.0,75.0,24.5,0.5,0.0,0.047,0.042,0.052,0.051,0.045,...,-0.007,-0.007,-0.007,-0.007,-0.006,-0.006,-0.006,-0.005,-0.005,-0.005
3,5.0,75.0,24.5,0.5,0.0,0.036,0.036,0.038,0.042,0.040,...,-0.006,-0.006,-0.006,-0.006,-0.004,-0.004,-0.004,-0.003,-0.003,-0.003
4,5.0,75.0,24.0,1.0,0.0,0.027,0.028,0.032,0.038,0.031,...,-0.004,-0.004,-0.004,-0.004,-0.002,-0.003,-0.002,-0.002,-0.002,-0.001
5,5.0,75.0,24.0,1.0,0.0,0.038,0.044,0.046,0.049,0.048,...,-0.005,-0.005,-0.005,-0.005,-0.003,-0.004,-0.003,-0.003,-0.003,-0.002
6,5.0,75.0,20.0,5.0,0.0,0.042,0.044,0.045,0.050,0.048,...,-0.008,-0.008,-0.008,-0.008,-0.006,-0.007,-0.006,-0.006,-0.006,-0.006
7,5.0,75.0,20.0,5.0,0.0,0.046,0.051,0.055,0.055,0.052,...,-0.007,-0.007,-0.007,-0.007,-0.005,-0.006,-0.005,-0.005,-0.005,-0.004
8,5.0,75.0,25.0,0.0,10.0,0.144,0.134,0.148,0.159,0.152,...,-0.005,-0.005,-0.005,-0.005,-0.004,-0.004,-0.003,-0.003,-0.003,-0.003
9,5.0,75.0,25.0,0.0,10.0,0.111,0.101,0.116,0.121,0.116,...,-0.009,-0.008,-0.009,-0.009,-0.008,-0.009,-0.007,-0.007,-0.007,-0.007
